# Nutrition Calculator  - Australian Food Composition Database

mission statement

#### Data sources
nutrition data: https://www.foodstandards.gov.au/science-data/monitoringnutrients/afcd 
<br> recommended intake data: https://www.myfooddata.com/articles/recommended-daily-intakes.php

To Do:
- Make units kiss
    - Check certain foods
- Add omega 6s properly
- Add nutrients back in with improved data imputation
    - Amino acids
    - Sugars
- Use classifier trained on recipe file to make appropraite food portions
    - Use this to adjust lin prog restrictions
- Do dot product of foods with DV
    - Adjust weights on nutrients
- How to interpret nutrient intakes
    - Daily? +/- certain amount given that you make it up in a day?
- Optimize lin prog w.r.t.
    - Protein
    - Fiber
    - Calories
- Make macro goals variable for function

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
sns.set()
%matplotlib inline

In [2]:
# Regex debugger #
pattern = r'\s?\n\(.+\)'
df = pd.DataFrame({'Lysine \n(mg)':[1,2]})
new_cols = df.columns.str.replace(pattern, '', regex=True)
print(new_cols)

Index(['Lysine'], dtype='object')


In [3]:
# Import food and nutrition data #

food = pd.read_excel('Australian_Food_Composition_Database/Food Details.xlsx') 
nutrient = pd.read_excel('Australian_Food_Composition_Database/Nutrient file.xlsx',
                         sheet_name='All solids & liquids per 100g')

## Need to drop percent fatty acid content, not necessary for calculation and data cleaning causes repition ##
nutrient = nutrient.drop(columns = ['Total polyunsaturated fatty acids, equated (%T)', 
                          'Total long chain omega 3 fatty acids, equated \n(%T)'])
## Import Daily Value (DV) ##

dv = pd.read_csv('daily value.csv', index_col=0)
dv.head()

,RDI,DV,UL
Nutrient,,,
Calories,2000,2000,NaN
Fat,NaN,78g,NaN
Saturated Fat,NaN,20g,NaN
Cholesterol,NaN,300mg,NaN
Carbs,130g,275g,NaN


In [4]:
## Need to drop percent content columns. Not relevant to goal and causing duplicate columns later on ##
print("Number of columns in df before drop:", len(nutrient.columns))

percent_pattern = r'\(%\w+\)'
percent_cols = nutrient.columns.str.contains(percent_pattern)
print("Number of columns with % in header:", percent_cols.sum())
nutrient = nutrient.drop(columns = nutrient.columns[percent_cols])
print("Number of columns after dropping %:", len(nutrient.columns))

gN_pattern = r'\/gN'
gN_cols = nutrient.columns.str.contains(gN_pattern)
print("Number of columns with /gN in header:", gN_cols.sum())
nutrient = nutrient.drop(columns = nutrient.columns[gN_cols])
print("Number of columns after dropping /gN:", len(nutrient.columns))

Number of columns in df before drop: 291
Number of columns with % in header: 69
Number of columns after dropping %: 222
Number of columns with /gN in header: 18
Number of columns after dropping /gN: 204


In [5]:
nutrient.head()

,Public Food Key,Classification,Food Name,"Energy with dietary fibre, equated \n(kJ)","Energy, without dietary fibre, equated \n(kJ)",Moisture (water) \n(g),Protein \n(g),Nitrogen \n(g),"Fat, total \n(g)",Ash \n(g),...,Leucine \n(mg),Lysine \n(mg),Methionine \n(mg),Phenylalanine \n(mg),Proline \n(mg),Serine \n(mg),Threonine \n(mg),Tyrosine \n(mg),Tryptophan \n(mg),Valine \n(mg)
0,F002258,31302.0,"Cardamom seed, dried, ground",1236,1012,8.3,10.8,1.72,6.7,5.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155,NaN
1,F002893,31302.0,"Chilli (chili), dried, ground",1280,1002,10.8,13.4,2.14,14.3,11.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69,NaN
2,F002963,31302.0,"Cinnamon, dried, ground",1004,579,10.6,4.0,0.64,1.2,3.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49,NaN
3,F002970,31302.0,"Cloves, dried, ground",1389,1118,9.9,6.0,0.96,13.0,5.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,NaN
4,F003190,31302.0,"Coriander seed, dried, ground",1344,1009,8.9,12.4,1.98,17.8,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178,NaN


## Data Cleaning

In [6]:
# Get rid of \n's from headers #
col_pattern = r'\s?\n\(.+\)'
nutr_cols = nutrient.columns.str.replace(col_pattern, '', regex=True)
new_cols = dict(zip(nutrient.columns, nutr_cols))
nutrient_clean = nutrient.rename(columns=new_cols)

# Change energy to calories #

fiber=True #toggle this to change how calories from fiber are calculated

if fiber==True:
    energy_cols = {'Energy with dietary fibre, equated': 'Calories'}
    nutrient_clean = nutrient_clean.rename(columns=energy_cols)
    nutrient_clean = nutrient_clean.drop(columns='Energy, without dietary fibre, equated')

else:
    energy_cols = {'Energy, without dietary fibre, equated': 'Calories'}
    nutrient_clean = nutrient_clean.rename(columns=energy_cols)
    nutrient_clean = nutrient_clean.drop(columns='Energy with dietary fibre, equated')

## Convert kJ to kCal ## 
nutrient_clean['Calories'] = nutrient_clean['Calories'].apply(lambda x: x/4.2)  #multiply kJ by this to get kcal

In [7]:
# Extract Units From Cols#

### Attempt 1 - Deleting###
"""The main plan is to delete units in table header. Because every unit in both datasets are metric, an inconsistency 
in units would manifest as a 3 orders of magnitude off which will be investigated in further analysis."""

# Get rid of (abbrev.)#
col_pattern = r' \(.+\)'
nutr_cols = nutrient_clean.columns.str.replace(col_pattern, '',regex=True)
new_cols = dict(zip(nutrient_clean.columns, nutr_cols))
nutrient_clean = nutrient_clean.rename(columns=new_cols)


In [8]:
np.asarray(nutrient_clean.columns)

array(['Public Food Key', 'Classification', 'Food Name', 'Calories',
       'Moisture', 'Protein', 'Nitrogen', 'Fat, total', 'Ash',
       'Total dietary fibre', 'Alcohol', 'Fructose', 'Glucose', 'Sucrose',
       'Maltose', 'Lactose', 'Galactose', 'Maltotrios', 'Total sugars',
       'Added sugars', 'Free sugars', 'Starch', 'Dextrin', 'Glycerol',
       'Glycogen', 'Inulin', 'Erythritol', 'Maltitol', 'Mannitol',
       'Xylitol', 'Maltodextrin', 'Oligosaccharides ', 'Polydextrose',
       'Raffinose', 'Stachyose', 'Sorbitol', 'Resistant starch',
       'Available carbohydrate, without sugar alcohols',
       'Available carbohydrate, with sugar alcohols', 'Acetic acid',
       'Citric acid', 'Fumaric acid', 'Lactic acid', 'Malic acid\n',
       'Oxalic acid', 'Propionic acid', 'Quinic acid', 'Shikimic acid',
       'Succinic acid', 'Tartaric acid', 'Aluminium', 'Antimony',
       'Arsenic', 'Cadmium', 'Calcium', 'Chromium', 'Chloride', 'Cobalt',
       'Copper', 'Fluoride', 'Iodine', '

In [9]:
# Need to change column names to be equivalent between food and DV #

## Vitamins ##
vitamins = {'Thiamin': 'Vitamin B1',
'Riboflavin': 'Vitamin B2',
'Niacin derived equivalents': 'Vitamin B3', 
'Pantothenic acid':'Vitamin B5',
'Pyridoxine':'Vitamin B6',
'Biotin': 'Vitamin B7',
'Total folates': 'Vitamin B9',
'Cobalamin': 'Vitamin B12',
'Vitamin A retinol equivalents':'Vitamin A',
'Vitamin D3 equivalents': 'Vitamin D'
           }
nutrient_clean = nutrient_clean.rename(columns=vitamins)

## Fat ##
nutrient_clean = nutrient_clean.rename(columns = {'Fat, total': 'Fat', 
                                                    'Total saturated fatty acids, equated': 'Saturated Fat',
                                                   'Total long chain omega 3 fatty acids, equated': 'Omega 3s'
                                                   })

### Adding omega 6s from manual Calc ###
### Assuming any polyunsaturated fat that isn't omega-3 is an omega-6
nutrient_clean['Omega 6s'] = round(nutrient_clean['Total polyunsaturated fatty acids, equated'] - nutrient_clean['Omega 3s']*0.001,2)

## Carbohydrates ##
carbs = {'Total sugars':'Sugar',
          'Total dietary fibre':'Fiber',
        'Available carbohydrate, with sugar alcohols':'Carbs'}
nutrient_clean = nutrient_clean.rename(columns=carbs)

## Misc Nutrients ##

misc_nutr = {'Cystine plus cysteine':'Cystine',
            }
nutrient_clean = nutrient_clean.rename(columns=misc_nutr)


### Investigating NaN's

In [10]:
# Look at nutrient NaNs and determine whether or not they represent 0's

## Using leucine as an example 
leucine = nutrient_clean['Leucine']
print('Number of foods with null leucine values:', leucine.isnull().sum(),
      '\nNumber of foods with non-null:', leucine.notnull().sum())

notnull_leucine = nutrient_clean[leucine.notnull()]
null_leucine = nutrient_clean[leucine.isnull()]

"""The vast majority of foods are missing amino acid data. It would require combining an entire dataset which would
involve matching food names accurately, which is outside of the scope of this project. Either food with missing data 
must be excluded, or we must exclude these nutrients from the fitting criteria. In order to preserve the most data
possible, I an doing to drop the nutrients instead of the foods. For similar reasons as listed below, amino acids
are subcategories of protein, which is included in the data set. I do believe the fitting of these nutrients, like
many others being excluded, to be important and should be subject of an expansion to this study."""

# Make NaNs in food data 0 #
# nutrient_values_only = nutrient_values_only.fillna(0)


## Find columns with mostly NaN's and drop

mostly_empty = []

for cols in nutrient_clean:
    if nutrient_clean[cols].isnull().sum() > nutrient_clean[cols].notnull().sum():
        mostly_empty.append(cols)
        
nutrient_clean = nutrient_clean.drop(columns=mostly_empty)

Number of foods with null leucine values: 1435 
Number of foods with non-null: 181


In [11]:
# Dropping carb types #

"""In a similar vein, carbohydrates are non-essential nutrients (meaning there are no essential carbohydrates), so the
carbohydrate composition is not important information. However, it is best to have complex carbohydrates instead of 
simple ones like fructose and starches. For now, the total carbohydrates and total sugar will behave as a good proxy
measurement. Furthermore, having too many carbohydrate fitting variables, and too few protein and fat, could bias the
output. For these reasons, all but these columns will be dropped. Including these nutrients with fat and amino acid 
compositions would serve as a valuable expansion of this model."""

carb_cols = ['Starch', 'Sucrose', 'Glucose', 'Fructose', 'Lactose', 'Maltose']
dv = dv.drop(index=carb_cols)
nutrient_clean = nutrient_clean.drop(columns=carb_cols)

## Drop ash column, not a relevant nutrient ##
nutrient_clean = nutrient_clean.drop(columns='Ash')

In [12]:
def shared_columns(df1, df2):
    
    """Finds which columns in df1 are and are not shared with df2"""
    
    df1_cols = df1.columns
    df2_cols = df2.columns
    
    shared_cols = []
    diff_cols = []
    
    for cols in df1_cols:
        if cols in df2_cols:
            shared_cols.append(cols)
        else:
            diff_cols.append(cols)
    
    return shared_cols, diff_cols

In [13]:
## Find DV columns that aren't equal to nutrient cols ##

dv_clean = dv.rename(index={'Lutein+zeazanthin': 'Lutein'})
nutr_cols = nutrient_clean.columns
dv_shared = shared_columns(dv_clean.transpose(), nutrient_clean)


"""This data does not exist in the food list df, therefore we will ignore these nutrients. Water is a 
nutrient that should be satisfied outside of food consumption. Therefore, it should be excluded
from the analysis. Furthermore, the remaining incongruent sets are non-essential amino acids which can
be produced in the human body, making it less important for consumption. Vitamin K and Choline
are essential in the human diet. However, the food data set does not have this included. This data could 
be imputed in the event of expanding the model for a more robust study."""

dv_clean = dv_clean.drop(index=dv_shared[1])
print("Dropping columns:", dv_shared[1])
dv_shared_dropped = shared_columns(dv_clean.transpose(), nutrient_clean)

print("Shared columns:", len(dv_shared_dropped[0]),"\nDifferent columns:", len(dv_shared_dropped[1]))



Dropping columns: ['Vitamin K', 'Choline', 'Lycopene', 'Lutein', 'Ash', 'Water', 'Galactose', 'Histidine', 'Threonine', 'Isoleucine', 'Lysine', 'Leucine', 'Methionine', 'Cystine', 'Phenylalanine', 'Tyrosine', 'Valine', 'Arginine', 'Alanine', 'Aspartic Acid', 'Glutamic Acid', 'Glycine', 'Proline', 'Serine', 'Hydroxyproline']
Shared columns: 33 
Different columns: 0


In [14]:
# Find nutrient columns that aren't in the DV columns #

nutr_cols = nutrient_clean.columns
nutr_shared = shared_columns(nutrient_clean, dv_clean.transpose())
print("Shared columns:", len(nutr_shared[0]),"\nDifferent columns:", len(nutr_shared[1]))

"""Much of the remaining dataset is anti-nutrients, subcategories of nutrients which are encapsulated by other
nutrients, and nutrients which have no recommended value. Currently, anti-nutrients are not in the scope of this
study but could be a valuable addition later. Therefore, I will drop all excess columns except the food name
column."""

food_name = nutrient_clean['Food Name']
print("Dropping excess nutrient columns", nutr_shared[1])

nutrient_values_only = nutrient_clean.drop(columns=nutr_shared[1])
nutr_shared = shared_columns(dv_clean.transpose(), nutrient_values_only)
print("Shared columns:", len(nutr_shared[0]),"\nDifferent columns:", len(nutr_shared[1]))
# print(nutr_shared[1])

Shared columns: 33 
Different columns: 47
Dropping excess nutrient columns ['Public Food Key', 'Classification', 'Food Name', 'Moisture', 'Nitrogen', 'Alcohol', 'Added sugars', 'Free sugars', 'Available carbohydrate, without sugar alcohols', 'Retinol', 'Alpha-carotene', 'Beta-carotene', 'Cryptoxanthin', 'Beta-carotene equivalents', 'Niacin', 'Niacin derived from tryptophan', 'Folate, natural', 'Folic acid', 'Dietary folate equivalents', 'Cholecalciferol', 'Ergocalciferol', '25-hydroxy cholecalciferol', '25-hydroxy ergocalciferol', 'Alpha tocopherol', 'C14', 'C15', 'C16', 'C17', 'C18', 'C22', 'C14:1', 'C16:1', 'C18:1', 'Total monounsaturated fatty acids, equated', 'C18:2w6', 'C18:3w3', 'C18:3w6', 'C20:2w6', 'C20:3w6', 'C20:4w6', 'C20:5w3', 'C22:5w3', 'C22:4w6', 'C22:6w3', 'Total polyunsaturated fatty acids, equated', 'Total trans fatty acids, imputed', 'Caffeine']
Shared columns: 33 
Different columns: 0


In [15]:
nutrient_values_only.head()

,Calories,Protein,Fat,Fiber,Sugar,Carbs,Calcium,Copper,Iodine,Iron,...,Vitamin B12,Vitamin B9,Vitamin C,Vitamin D,Vitamin E,Saturated Fat,Omega 3s,Cholesterol,Tryptophan,Omega 6s
0,294.285714,10.8,6.7,28.0,4.4,34.4,383,NaN,0.0,13.97,...,0.0,3,21,0.0,2.85,2.20,0.000,0,155,1.39
1,304.761905,13.4,14.3,34.8,7.2,14.9,330,NaN,2.3,17.30,...,0.0,28,1,0.0,38.14,2.06,0.000,0,69,6.69
2,239.047619,4.0,1.2,53.1,2.2,27.5,1002,NaN,0.1,8.32,...,0.0,6,4,0.0,2.32,0.62,0.000,0,49,0.12
3,330.714286,6.0,13.0,33.9,2.4,31.6,632,NaN,50.7,11.83,...,0.0,25,0,0.0,8.82,5.49,266.581,0,30,4.76
4,320.000000,12.4,17.8,41.9,2.4,8.4,709,NaN,2.3,16.32,...,0.0,0,21,0.0,0.91,1.03,0.000,0,178,1.82


In [16]:
# Extract Units From DV #

unit_pattern = r'\d+(\w+)'
number_pattern = r'(\d+.?\d+)'
dv_clean['RDI units'] = dv_clean['RDI'].str.extract(unit_pattern)
dv_clean['RDI'] = dv_clean['RDI'].str.extract(number_pattern)
dv_clean['DV units'] = dv_clean['DV'].str.extract(unit_pattern)
dv_clean['DV'] = dv_clean['DV'].str.extract(number_pattern)

# Make numeric entries numeric type
dv_clean['DV'] = pd.to_numeric(dv_clean['DV'])
dv_clean['RDI'] = pd.to_numeric(dv_clean['RDI'])

In [17]:
"""Let's combine the entries in RDI and DV to create a daily value with few NaNs, taking the highest value
Note: isnan() is vectorized so I might be able to get rid of loop."""

dv_col = dv_clean['DV']
rdi_col = dv_clean['RDI']
dv_rdi_combined = []

for dv, rdi in zip(dv_col,rdi_col):
    if np.isnan(dv) and not np.isnan(rdi):
        dv_rdi_combined.append(rdi)
    
    elif not np.isnan(dv) and np.isnan(rdi):
        dv_rdi_combined.append(dv)
    
    elif np.isnan(dv) and np.isnan(rdi):
        dv_rdi_combined.append(dv)
        
    elif not np.isnan(dv) and not np.isnan(rdi):
        dv_rdi_combined.append(max(dv,rdi))

dv_clean['DV and RDI, Highest'] = dv_rdi_combined

### Impute Missing Vitamin B5 Value
Sources: <br>
Vitamin B5: https://health.clevelandclinic.org/vitamin-b5-pantothenic-acid <br>

In [18]:
# Impute DV/RDI  #
dv_clean.loc['Vitamin B5', 'DV and RDI, Highest'] = 5.0
dv_clean.loc['Vitamin B5', 'DV units'] = 'mg'


In [19]:
dv_clean['DV and RDI, Highest']

Nutrient
Calories         2000.0
Fat                78.0
Saturated Fat      20.0
Cholesterol       300.0
Carbs             275.0
Fiber              38.0
Sugar              50.0
Protein            56.0
Vitamin A        3000.0
Vitamin B1          1.2
Vitamin B2          1.3
Vitamin B3         16.0
Vitamin B5          5.0
Vitamin B6          1.7
Vitamin B9        400.0
Vitamin B12         2.4
Vitamin C          90.0
Vitamin D          20.0
Vitamin E          15.0
Calcium          1300.0
Copper              0.9
Iodine            150.0
Iron               18.0
Magnesium         420.0
Manganese           2.3
Phosphorus       1250.0
Potassium        4700.0
Selenium           55.0
Sodium           2300.0
Zinc               11.0
Omega 3s            1.6
Omega 6s           17.0
Tryptophan        280.0
Name: DV and RDI, Highest, dtype: float64

### Sanity Check

In [20]:
a = {}
for nut in nutrient_clean.columns:
    if nut in a:
        a[nut] +=1
    else:
        a[nut] = 1

In [21]:
b = 0
for keys, vals in a.items():
    if vals > 1:
        print('Repeat column')
        b+=1
if b == 0:
    print('No repeat columns')

No repeat columns


In [22]:
nutrient_values_only.isnull().sum()

Calories           0
Protein            0
Fat                0
Fiber              0
Sugar              0
Carbs              0
Calcium            0
Copper           361
Iodine             0
Iron               0
Magnesium          0
Manganese        392
Phosphorus         0
Potassium          0
Selenium           0
Sodium             0
Zinc               0
Vitamin A          0
Vitamin B1         0
Vitamin B2         0
Vitamin B3         0
Vitamin B5       574
Vitamin B6         0
Vitamin B12        0
Vitamin B9         0
Vitamin C          0
Vitamin D          0
Vitamin E          0
Saturated Fat      0
Omega 3s           0
Cholesterol        0
Tryptophan         0
Omega 6s           0
dtype: int64

## Find random solutions to the inequality

In [146]:
# Fill in NaNs with 0s and move on#
nutrient_values_only = nutrient_values_only.fillna(0.0)

In [24]:
# Set the inequalities #
"""Note: For now we will ignore dietary cholesterol due to it mostly being seen as benign."""
"""Note: There are few upper limits, so they are set arbitrarily high unless otherwise specified.""" 

## Set mins ##
v_min = dv_clean['DV and RDI, Highest'].copy() #most values are a minimum, start here
v_min['Sugar'] = 0 #Sugar in DV is a max
v_min['Saturated Fat'] = 0 #Sat Fat in DV is a max
v_min['Calories'] = 1700 #Caution on the lower side
v_min['Cholesterol'] = 0

## Set carbs and fat inequality to be +/- 20% RDI/DV
v_min['Carbs'] = dv_clean.loc['Carbs', 'DV and RDI, Highest']*.80
v_min['Fat'] = dv_clean.loc['Carbs', 'DV and RDI, Highest']*.80

## Set maxs ##
v_max = dv_clean['DV and RDI, Highest'].copy()*100 #most values are a minimum, set max to be arbitrarily high
v_max['Sugar'] = dv_clean.loc['Sugar', 'DV and RDI, Highest'] # Sugar in DV is a max
v_max['Saturated Fat'] =  dv_clean.loc['Saturated Fat', 'DV and RDI, Highest']# Sat Fat in DV is a max
v_max['Calories'] = 2100 # A little over is OK


## Set carbs and fat inequality to be +/- 20% RDI/DV
v_max['Carbs'] = dv_clean.loc['Carbs', 'DV and RDI, Highest']*1.20
v_max['Fat'] = dv_clean.loc['Carbs', 'DV and RDI, Highest']*1.20


In [52]:
# Generate random solutions #
"""This approach is indefinite and takes forever. Also it will likely produce too many """
def generate_random_solutions(F, v_min, v_max, a_max, precision=2, num_samples=1):
    """Generates a random a vector until one satisfies v_min < Fa < v_max"""
    num_vars = F.shape[1]  # Number of variables (components of vector a)
    solutions = []
    for _ in range(num_samples):
        valid = False
        attempt_count=0
        while not valid:
            # Generate a random vector a within the given bounds
            a = np.array([round(np.random.uniform(0, a_max),precision) for i in range(num_vars)])
     
            # Check if random guess satisfies inequality
            if np.all(np.dot(F, a) < v_max) and np.all(np.dot(F,a) > v_min):
                valid = True
                solutions.append(a)
            elif attempt_count < 1e6:
                attempt_count += 1
            else:
                print(f'No solution found after {attempt_count} attempts')
                valid = True
                
    
    return solutions

In [53]:
# Generate a random food and add back food names #
random_food_solution = generate_random_solutions(F=nutrient_values_only.transpose(), v_min=v_min, v_max=v_max, a_max=5000)
random_food_solution
# random_food = pd.DataFrame({food_name:random_food_solutions})
# random_food.head()

KeyboardInterrupt: 

## Using least squares to find optimal solution to the linear equation

In [159]:
## Sanity check, lets find a solution using select part of ##
v_target = dv_clean['DV and RDI, Highest'].copy()
v_target_reduced = np.asarray(v_target.loc[['Calories', 'Fat', 'Carbs','Protein']])
food_df_reduced = nutrient_values_only[['Calories', 'Fat', 'Carbs','Protein']]
food_matrix_reduced = np.asarray(food_df_reduced.transpose())

food, res = nnls(A=food_matrix_reduced, b=v_target_reduced)
for f in food:
    nonzero = []
    if f > 0.01:
        nonzero.append(f)
nonzero


[]

In [160]:
v_target_reduced

array([2000.,   78.,  275.,   56.])

In [163]:
a = np.asarray(food_df_reduced.loc[0:5]).transpose()
ax = np.dot(a, [1,2,3,4,4,6])
v_target_reduced - ax

array([-4769.52380952,  -217.9       ,  -233.9       ,  -174.        ])

Nutrient
Calories    2000.0
Fat           78.0
Carbs        275.0
Protein       56.0
Name: DV and RDI, Highest, dtype: float64

In [111]:
from scipy.optimize import nnls
import random
def rand_lstsq_solution(F, v_target, num_samples,seed=False):
    if seed == True:
        random.seed(42)
    sample = [i for i in range(F.shape[1])]
    rand_ind = random.sample(sample, num_samples)
    rand_names = food_name[rand_ind]
    rand_solution, res = nnls(A=F[:,rand_ind], b=v_target)
    
    return [rand_names, rand_solution, res]


    


In [134]:
# Generate random foods and find closest solution

v_target = dv_clean['DV and RDI, Highest'].copy()
food_matrix = np.asarray(nutrient_values_only.transpose())

food_amounts = rand_lstsq_solution(food_matrix, np.asarray(v_target), 1616)
today_you_eat = pd.DataFrame({'Food': food_amounts[0], 'Amount (g)': food_amounts[1][0]})


In [135]:
food_amounts[2]

6069.833766728892

In [136]:
today_you_eat

,Food,Amount (g)
1165,"Sausages & vegetables, canned",0.0
616,"Orange, peeled, raw",0.0
831,"Veal, diced, lean, raw",0.0
1122,"Kangaroo, steak, as purchased, raw",0.0
0,"Cardamom seed, dried, ground",0.0
...,...,...
628,"Pear, green skin, unpeeled, raw",0.0
1434,"Capsicum, red, fresh, fried, no added fat",0.0
387,"Paste, soybean",0.0
536,"Fat, solid, vegetable oil based",0.0


In [38]:
# ## Solve ##
from scipy.linalg import lstsq
rdi_limit = np.asarray(dv_clean['RDI'])
food_matrix = np.asarray(nutrient_values_only.transpose())

food_solution = lstsq(a=food_matrix, b=rdi_limit)
food_solution

## Using linear programming to generate a set of feasible solutions

The tables above leave us with a linear equation of the form:
$$a_{1}f_{1} + ... + a_{m}f_{m} = v_{T}$$ 
where, $$a_{k} \in \mathbb{R}, f_{k} \in V$$ 
and, $$k \in {0,...,m}$$

Here the daily value required is represented by the vector $v_T$, with entries in the vector being daily value nutrient limits.

The list of vectors, $f$, represent the nutrient profile of a given food, with its $k^{th}$ element being the $k^{th}$ nutrient in $v_T$.

The scalars, $a$, are the amounts, or portion size, of the foods in $f$ that satisfy the equation above. This equation can be written in its matrix form:

$$Fa = v_T$$

By solving the equation for the amounts $a$, we can get a list of foods that satisfy our days nutrition needs. Thus, satisfying the goal of this project.

However, with foods this will rarely result in an exact solution. Instead, we want to find a set of solutions that satisfy the inequality,

\begin{array}{r@{\quad}r@{\quad}>{\dotfill}l}
a_{1}f_{1,1} + \ldots + a_{m}f_{m,1} & > & v_{T, 1} \\
\ &  \vdots \\
a_{1}f_{1,n} + \ldots + a_{m}f_{m,n} & > & v_{T, n}
\end{array}

To do this, we can use a method called "Linear Programming." We will create a feasible region using these inequalities and explore it. Furthermore, we can explore this region to optimize for different nutrients, like minimizing calories or maximizing protein.

# Delete cell below and finish data cleaning

In [38]:
# Limit food amounts such that you can't have less than 10g
"""Binning the amounts such that if  0<a<0.1 else a=a is a non-trivial task. Will try to implement this later."""
a_lim = [0.1 for _ in nutrient_values_only.index]
print(len(a_lim))

1616


In [39]:
def matrix_mult(M,x):
    return np.dot(M,x)

In [40]:
# v_min

In [41]:
# b_ub

In [42]:
# M.shape

In [43]:
## Try optimize for calories ##

from scipy.optimize import linprog
nutrient_values_only_no_cal = nutrient_values_only.drop(columns='Calories')
v_min_no_cal = v_min.drop(index='Calories')
v_max_no_cal = v_max.drop(index='Calories')

# Define the matrix M 
M = np.asarray(nutrient_values_only_no_cal).transpose()

# Define the vectors v_min and v_max
v_min = np.asarray(v_min_no_cal)
v_max = np.asarray(v_max_no_cal)

# Number of variables (length of vector a)
n = M.shape[1]

# Define the dummy objective function coefficients (all zeros)
c = np.asarray(nutrient_values_only['Calories']).transpose()

# Define the inequality constraint matrix and bounds
A_ub = np.vstack([M, -M])
b_ub = np.hstack([v_max, -v_min])

# Solve the linear programming problem to find any feasible solution
res = linprog(c, A_ub=A_ub, b_ub=b_ub, method='highs')

# Check if a feasible solution was found and output the result
if res.success:
    print('Feasible solution found:', res.x)
else:
    print('No feasible solution found.')

No feasible solution found.


In [44]:
from scipy.optimize import linprog

# Define the matrix M 
M = np.asarray(nutrient_values_only).transpose()

# Define the vectors v_min and v_max
v_min = np.asarray(v_min)
v_max = np.asarray(v_max)

# Number of variables (length of vector a)
n = M.shape[1]

# Define the dummy objective function coefficients (all zeros)
c = np.zeros(n)

# Define the inequality constraint matrix and bounds
A_ub = np.vstack([M, -M])
b_ub = np.hstack([v_max, -v_min])

# Solve the linear programming problem to find any feasible solution
res = linprog(c, A_ub=A_ub, b_ub=b_ub, method='highs')

# Check if a feasible solution was found and output the result
if res.success:
    print('Feasible solution found:', res.x)
else:
    print('No feasible solution found.')

ValueError: Invalid input for linprog: b_ub must be a 1-D array; b_ub must not have more than one non-singleton dimension and the number of rows in A_ub must equal the number of values in b_ub

In [ ]:
c.shape

In [ ]:
b_ub.shape

In [ ]:
A_ub.shape

In [ ]:
from scipy.optimize import linprog

# Define the coefficients of the dummy objective function (all zeros)
c = np.zeros(2)

# Define the inequality constraint matrix
A_ub = np.array([[1, 2],
                 [3, 1]])

# Define the upper bounds of the inequality constraints
b_ub = np.array([20, 30])

# Define the bounds for each variable
x0_bounds = (0, None)  # x1 >= 0
x1_bounds = (0, None)  # x2 >= 0

def generate_feasible_solutions(A_ub, b_ub, bounds, num_samples=10):
    feasible_solutions = []
    for _ in range(num_samples):
        # Perturb the bounds slightly to explore different feasible regions
        perturbation = np.random.uniform(-1, 1, size=b_ub.shape)
        b_ub_perturbed = b_ub + perturbation

        # Solve the LP problem
        res = linprog(c, A_ub=A_ub, b_ub=b_ub_perturbed, bounds=bounds, method='highs')

        if res.success:
            feasible_solutions.append(res.x)
    
    return feasible_solutions

# Generate a set of feasible solutions
feasible_solutions = generate_feasible_solutions(A_ub, b_ub, [x0_bounds, x1_bounds], num_samples=10)

print("Feasible solutions:")
for i, solution in enumerate(feasible_solutions, 1):
    print(f"Solution {i}: {solution}")